In [1]:
import librosa
import librosa.feature
import librosa.display
import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils.np_utils import to_categorical
import os
import time
from sklearn import svm


C:\Users\XJX\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#找到所有乐曲中信号数量最少的，防止出现特征长度不一致，与上面的数据一致，保障特征数量相等。
miny = 10000000
name = os.listdir(r"D:\弦鸣乐器音乐库\正式版")
num_of_class = len(name)
for num in range(100):
    r = str(num)
    r = r.zfill(3)
    for i in range(num_of_class):
        path = r"D:\弦鸣乐器音乐库\正式版\\" + name[i] + '\\' + '%s.wav' %r
        y, _ = librosa.load(path)
        if len(y)<miny:
            miny=len(y)
for num in range(50):
    r = str(num)
    r = r.zfill(3)
    for i in range(num_of_class):
        path = r"D:\弦鸣乐器音乐库\正式版测试集\\" + name[i] + '\\' + '%s.wav' %r
        y, _ = librosa.load(path)
        if len(y)<miny:
            miny=len(y)

print(len(y))

661500


In [3]:
num_of_class = len(name)

In [69]:
def extract_features_song(y,n):
    n_fft = 512 
    hop_length = 256
    #mfcc特征，输出的mfcc是一列特征，30s的片段分帧有偏差，所以一致取前25000个。
    #mfcc特征的排列顺序是每一帧的横向输出，而后下一帧的所有特征贴在上一帧的右侧。
    #下面的特征同理。
    mfcc = librosa.feature.mfcc(y)
    mfcc /= np.amax(np.absolute(mfcc))
    mfcc = np.ndarray.flatten(mfcc)
    feature = mfcc
    
    return feature

In [4]:
#y是信号特征，n是将30s片段分成的份数，以下所有分帧以2048为帧长，512为帧移。
def extract_features_song(y,n):
    n_fft = 512 
    hop_length = 256
    #mfcc特征，输出的mfcc是一列特征，30s的片段分帧有偏差，所以一致取前25000个。
    #mfcc特征的排列顺序是每一帧的横向输出，而后下一帧的所有特征贴在上一帧的右侧。
    #下面的特征同理。

    spectral_centroid = librosa.feature.spectral_centroid(y, n_fft = n_fft, hop_length = hop_length)[0]
    spectral_centroid /= np.amax(np.absolute(spectral_centroid))
    feature = spectral_centroid

    spectral_bandwidth = librosa.feature.spectral_bandwidth(y, n_fft = n_fft, hop_length = hop_length)[0]
    spectral_bandwidth /= np.amax(np.absolute(spectral_bandwidth))
    feature = np.append(feature,spectral_bandwidth)

    spectral_contrast = librosa.feature.spectral_contrast(y, n_fft = n_fft, hop_length = hop_length)[0]
    spectral_contrast /= np.amax(np.absolute(spectral_contrast))
    feature = np.append(feature,spectral_contrast)

    spectral_rolloff = librosa.feature.spectral_rolloff(y, n_fft = n_fft, hop_length = hop_length)[0]
    spectral_rolloff /= np.amax(np.absolute(spectral_rolloff))
    feature = np.append(feature,spectral_rolloff)

    spectral_flatness = librosa.feature.spectral_flatness(y, n_fft = n_fft, hop_length = hop_length)[0]
    spectral_flatness /= np.amax(np.absolute(spectral_flatness))
    feature = np.append(feature,spectral_flatness)

    rms = librosa.feature.rms(y, frame_length = n_fft, hop_length = hop_length)[0]
    rms /= np.amax(np.absolute(rms))
    feature = np.append(feature,rms)

    zero_all = sum(librosa.zero_crossings(y, pad=False))/len(y)
    feature = np.append(feature,zero_all)

    #这里的返回向量是横向的，一行若干列，例如shape=(1,3000)，所有特征都是连接的，mfcc在最前，过零率在最后
    return feature

In [280]:
#y是信号特征，n是将30s片段分成的份数，以下所有分帧以2048为帧长，512为帧移。
def extract_features_song(y,n):
    n_fft = 512 
    hop_length = 256
    #mfcc特征，输出的mfcc是一列特征，30s的片段分帧有偏差，所以一致取前25000个。
    #mfcc特征的排列顺序是每一帧的横向输出，而后下一帧的所有特征贴在上一帧的右侧。
    #下面的特征同理。
    mfcc = librosa.feature.mfcc(y)
    mfcc /= np.amax(np.absolute(mfcc))
    mfcc = np.ndarray.flatten(mfcc)
    feature = mfcc

    spectral_centroid = librosa.feature.spectral_centroid(y, n_fft = n_fft, hop_length = hop_length)[0]
    spectral_centroid /= np.amax(np.absolute(spectral_centroid))
    feature = np.append(feature,spectral_centroid)

    spectral_bandwidth = librosa.feature.spectral_bandwidth(y, n_fft = n_fft, hop_length = hop_length)[0]
    spectral_bandwidth /= np.amax(np.absolute(spectral_bandwidth))
    feature = np.append(feature,spectral_bandwidth)

    spectral_contrast = librosa.feature.spectral_contrast(y, n_fft = n_fft, hop_length = hop_length)[0]
    spectral_contrast /= np.amax(np.absolute(spectral_contrast))
    feature = np.append(feature,spectral_contrast)

    spectral_rolloff = librosa.feature.spectral_rolloff(y, n_fft = n_fft, hop_length = hop_length)[0]
    spectral_rolloff /= np.amax(np.absolute(spectral_rolloff))
    feature = np.append(feature,spectral_rolloff)

    spectral_flatness = librosa.feature.spectral_flatness(y, n_fft = n_fft, hop_length = hop_length)[0]
    spectral_flatness /= np.amax(np.absolute(spectral_flatness))
    feature = np.append(feature,spectral_flatness)

    rms = librosa.feature.rms(y, frame_length = n_fft, hop_length = hop_length)[0]
    rms /= np.amax(np.absolute(rms))
    feature = np.append(feature,rms)

    zero_all = sum(librosa.zero_crossings(y, pad=False))/len(y)
    feature = np.append(feature,zero_all)

    #这里的返回向量是横向的，一行若干列，例如shape=(1,3000)，所有特征都是连接的，mfcc在最前，过零率在最后
    return feature

In [5]:
#训练集的数据提取
def generate_train_features_and_labels():
    all_features = []
    all_labels = []
    all_number = []
    #获得该地址下内文件名称，这里是每个分类的名称
    genres = os.listdir(r"D:\弦鸣乐器音乐库\正式版")
    for genre in genres:
        #获得每个分类下，100个片段的地址名称
        sound_files = glob.glob(r"D:\弦鸣乐器音乐库\正式版\\"+genre+"\\*.wav")
        print('正在进行 %d 首训练集 %s 歌曲的特征提取...' % (len(sound_files), genre))
        #number代表的是当前小片段所处大片段的编号，文件后缀000，001，088这种
        number = 0
        for f in sound_files:
            #y是读取的信号
            y, _ = librosa.load(f)
            #将y的长度一致化
            y = y[:miny]
            #i从0到n-1，这里是按照每个片段分开读取特征
            for i in range(n):
                #提取特征中，输入到函数中的是y的片段
                features = extract_features_song(y[int(len(y)/n*i):int(len(y)/n*(i+1))],n)
                #features返回的向量为行向量，这里的append是把新的一行粘贴到原有的下面一行
                all_features.append(features)
                #genre是一个单独的字符串，append同样是往下面一行粘贴
                all_labels.append(genre)
                #number是数字，粘贴后事一维向量
                all_number.append(number)
            number += 1
    #这里将all_labels转化成数字，从0开始按顺序递增，这个顺序和直接读取文件，上一个运算格里的name[i]中的i是一样的。
    label_uniq_ids, label_row_ids = np.unique(all_labels, return_inverse=True)
    label_row_ids = label_row_ids.astype(np.int32, copy=False)
    #转换成onehot编码，0对应的是(1,0,0...)以此类推
    onehot_labels = to_categorical(label_row_ids, len(label_uniq_ids))
    #将一位向量reshape成二维方便后面操作
    all_number = np.reshape(all_number,[len(all_number),1])
    
    #返回特征、标签、及标号
    return np.stack(all_features), onehot_labels, all_number

#提取测试集的特征
def generate_test_features_and_labels():
    all_features = []
    all_labels = []
    all_number = []
    #获得该地址下内文件名称，这里是每个分类的名称
    genres = os.listdir(r"D:\弦鸣乐器音乐库\正式版测试集")
    for genre in genres:
        #获得每个分类下，100个片段的地址名称
        sound_files = glob.glob(r"D:\弦鸣乐器音乐库\正式版测试集\\"+genre+"\\*.wav")
        print('正在进行 %d 首训练集 %s 歌曲的特征提取...' % (len(sound_files), genre))
        #number代表的是当前小片段所处大片段的编号，文件后缀100，101，138这种，这里接在上面的训练集之后，从100开始。
        number = 100
        for f in sound_files:
            #y是读取的信号
            y, _ = librosa.load(f)
            #将y的长度一致化
            y = y[:miny]
            #i从0到n-1，这里是按照每个片段分开读取特征
            for i in range(n):
                #提取特征中，输入到函数中的是y的片段
                features = extract_features_song(y[int(len(y)/n*i):int(len(y)/n*(i+1))],n)
                #features返回的向量为行向量，这里的append是把新的一行粘贴到原有的下面一行
                all_features.append(features)
                #genre是一个单独的字符串，append同样是往下面一行粘贴
                all_labels.append(genre)
                #number是数字，粘贴后事一维向量
                all_number.append(number)
            number += 1
    #这里将all_labels转化成数字，从0开始按顺序递增，这个顺序和直接读取文件，上一个运算格里的name[i]中的i是一样的。
    label_uniq_ids, label_row_ids = np.unique(all_labels, return_inverse=True)
    label_row_ids = label_row_ids.astype(np.int32, copy=False)
    #转换成onehot编码，0对应的是(1,0,0...)以此类推
    onehot_labels = to_categorical(label_row_ids, len(label_uniq_ids))
    #将一位向量reshape成二维方便后面操作
    all_number = np.reshape(all_number,[len(all_number),1])
    
    #返回特征、标签、及标号
    return np.stack(all_features), onehot_labels, all_number

In [281]:
#先不分段，看整体分类结果
n = 30

In [282]:
features_train1, labels_train1, numbers_train1 = generate_train_features_and_labels()

正在进行 100 首训练集 二胡 歌曲的特征提取...
正在进行 100 首训练集 古筝 歌曲的特征提取...
正在进行 100 首训练集 吉他 歌曲的特征提取...
正在进行 100 首训练集 大提琴 歌曲的特征提取...
正在进行 100 首训练集 小提琴 歌曲的特征提取...
正在进行 100 首训练集 琵琶 歌曲的特征提取...
正在进行 100 首训练集 西塔尔琴 歌曲的特征提取...
正在进行 100 首训练集 钢琴 歌曲的特征提取...


In [283]:
features_test1, labels_test1, numbers_test1 = generate_test_features_and_labels()

正在进行 50 首训练集 二胡 歌曲的特征提取...
正在进行 50 首训练集 古筝 歌曲的特征提取...
正在进行 50 首训练集 吉他 歌曲的特征提取...
正在进行 50 首训练集 大提琴 歌曲的特征提取...
正在进行 50 首训练集 小提琴 歌曲的特征提取...
正在进行 50 首训练集 琵琶 歌曲的特征提取...
正在进行 50 首训练集 西塔尔琴 歌曲的特征提取...
正在进行 50 首训练集 钢琴 歌曲的特征提取...


In [284]:
#输出特征维度，保证准确度
print(np.shape(features_train1))
print(np.shape(labels_train1))
print(np.shape(numbers_train1))

print(np.shape(features_test1))
print(np.shape(labels_test1))
print(np.shape(numbers_test1))

#将特征合并到一起，方便后面随机排序
alldata_train1 = np.column_stack((features_train1, labels_train1, numbers_train1))
alldata_test1 = np.column_stack((features_test1, labels_test1, numbers_test1))

(24000, 1377)
(24000, 8)
(24000, 1)
(12000, 1377)
(12000, 8)
(12000, 1)


In [285]:
#测试集的编号及数量
test_set = np.arange(100,150)
num_of_test = len(test_set)

#去除NAN项，（实际上n=1或n=12均不存在）
df = pd.DataFrame(alldata_train1)
data_without_NaN = df.dropna(axis=0)
alldata_train = np.array(data_without_NaN)

df = pd.DataFrame(alldata_test1)
data_without_NaN = df.dropna(axis=0)
alldata_test = np.array(data_without_NaN)

#将训练集和测试集分类
train = []
test = []
#这里的判别方式是，alldata里的最后一列，即数字标号如果和test_set中相等，则放入测试集中
for i in range(len(alldata_train)):
    if sum(alldata_train[i,-1] == test_set) > 0:
        test.append(alldata_train[i,:])
    else:
        train.append(alldata_train[i,:])
        
for i in range(len(alldata_test)):
    if sum(alldata_test[i,-1] == test_set) > 0:
        test.append(alldata_test[i,:])
    else:
        train.append(alldata_test[i,:])
        

test = np.array(test)
train = np.array(train)

np.random.shuffle(train)

#从训练集和测试集的数量多少能看到上面分类是否准确，前两个输出应为4：1关系
print(np.shape(train))
print(np.shape(test))

#将之前合并的特征、标签、标号拆分出来，标号是最后一列，这里标签是最后10列，（若是8分类就是8列）
train_input = train[:,:-num_of_class-1]
train_labels = train[:,-num_of_class-1:-1]
train_numbers = train[:,-1]
train_numbers = train_numbers.reshape([len(train_numbers),1])

test_input = test[:,:-num_of_class-1]
test_labels = test[:,-num_of_class-1:-1]
test_numbers = test[:,-1]
test_numbers = test_numbers.reshape([len(test_numbers),1])

print(np.shape(train_input))
print(np.shape(train_labels))
print(np.shape(train_numbers))

(24000, 1386)
(12000, 1386)
(24000, 1377)
(24000, 8)
(24000, 1)


In [286]:
#同样的使用svm算法，经测试三次多项式判别为最佳，另外，svm不能使用onehot编码
begintime_poly = time.clock()
clf = svm.SVC(C=0.8, kernel='poly',degree=3, gamma=20, decision_function_shape='none')
clf.fit(train_input, [np.argmax(train_labels)for train_labels in train_labels])
endtime_poly = time.clock()

In [287]:
print('多项式核函数训练花费时间',endtime_poly-begintime_poly)

多项式核函数训练花费时间 240.7120894362124


In [288]:
train_X = train_input
train_Y = [np.argmax(train_labels)for train_labels in train_labels]
test_X = test_input
test_Y = [np.argmax(test_labels)for test_labels in test_labels]
#转出y_hat 与y_real后与上述操作相同
y_hat = clf.predict(test_X)
y_real = test_Y

In [289]:
predict = 0
for i in range(len(y_real)):
    if y_hat[i]==y_real[i]:
        predict+=1
print ('训练集准确度',predict/len(y_real)) 

训练集准确度 0.8525833333333334


In [290]:
#y_numbers是一个大矩阵，行数是测试大片段数量(20)乘分类数量(10)，列数与分类数量相同（10）
#其前20行是类型0的20个选取的大片段被分为了某一类的数量，如（5，0，0，0，4，0，1，0。。。）
y_numbers = np.zeros([num_of_test*num_of_class,num_of_class])
for i in range(len(y_hat)):
    for j in range(num_of_test):
        #test_numbers是测试的小片段所属的标号，test_set是20个被选为测试集的片段的标号
        if test_numbers[i] == test_set[j]:
            #如果上面两个标号相等，则将真实行，判定的类别的列上+1
            y_numbers[y_real[i]*num_of_test+j,y_hat[i]] += 1

In [291]:
#yy_hat是将y_numbers里每行最大的那个数字所在的位置输出出来，也就是大片段的预测值
yy_hat = [np.argmax(y_numbers)for y_numbers in y_numbers]
#按照上一运算格中的排序方式，大片段真实值yy_real应该按下述方式构造（0，0。。0，1，1。。，1，2。。。）
yy_real= np.zeros(num_of_test*num_of_class)
for i in range(num_of_class):
    for j in range(num_of_test):
        yy_real[i*num_of_test+j] = i
#yy_number是大片段对比正确的数量
yy_number = 0
for i in range(len(yy_real)):
    if yy_hat[i]==yy_real[i]:
        yy_number+=1
print('最终分类正确率是',yy_number/len(yy_real))

最终分类正确率是 0.9725


In [292]:
#给出错误的序号及判定
for i in range(len(yy_hat)):
    if yy_hat[i] != yy_real[i]:
        print(name[int(yy_real[i])],test_set[i%num_of_test],'被判定成了',name[int(yy_hat[i])])

二胡 130 被判定成了 钢琴
吉他 146 被判定成了 大提琴
小提琴 100 被判定成了 大提琴
小提琴 101 被判定成了 大提琴
小提琴 102 被判定成了 大提琴
小提琴 107 被判定成了 二胡
小提琴 138 被判定成了 大提琴
小提琴 142 被判定成了 古筝
琵琶 132 被判定成了 古筝
钢琴 107 被判定成了 古筝
钢琴 121 被判定成了 小提琴


In [293]:
test_matrix = np.zeros([num_of_class,num_of_class])
for i in range(len(yy_hat)):
        test_matrix[int(yy_real[i]),int(yy_hat[i])] += 1
print(test_matrix)

[[49.  0.  0.  0.  0.  0.  0.  1.]
 [ 0. 50.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. 49.  1.  0.  0.  0.  0.]
 [ 0.  0.  0. 50.  0.  0.  0.  0.]
 [ 1.  1.  0.  4. 44.  0.  0.  0.]
 [ 0.  1.  0.  0.  0. 49.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. 50.  0.]
 [ 0.  1.  0.  0.  1.  0.  0. 48.]]


In [294]:
#同样的使用svm算法，经测试三次多项式判别为最佳，另外，svm不能使用onehot编码
begintime_linear = time.clock()
clf = svm.SVC(C=0.8, kernel='linear',degree=3, gamma=20, decision_function_shape='none')
clf.fit(train_input, [np.argmax(train_labels)for train_labels in train_labels])
endtime_linear = time.clock()

In [295]:
print('线性核函数训练花费时间',endtime_linear-begintime_linear)

线性核函数训练花费时间 233.17893016401285


In [296]:
train_X = train_input
train_Y = [np.argmax(train_labels)for train_labels in train_labels]
test_X = test_input
test_Y = [np.argmax(test_labels)for test_labels in test_labels]
#转出y_hat 与y_real后与上述操作相同
y_hat = clf.predict(test_X)
y_real = test_Y

In [297]:
predict = 0
for i in range(len(y_real)):
    if y_hat[i]==y_real[i]:
        predict+=1
print ('训练集准确度',predict/len(y_real)) 

训练集准确度 0.81075


In [298]:
#y_numbers是一个大矩阵，行数是测试大片段数量(20)乘分类数量(10)，列数与分类数量相同（10）
#其前20行是类型0的20个选取的大片段被分为了某一类的数量，如（5，0，0，0，4，0，1，0。。。）
y_numbers = np.zeros([num_of_test*num_of_class,num_of_class])
for i in range(len(y_hat)):
    for j in range(num_of_test):
        #test_numbers是测试的小片段所属的标号，test_set是20个被选为测试集的片段的标号
        if test_numbers[i] == test_set[j]:
            #如果上面两个标号相等，则将真实行，判定的类别的列上+1
            y_numbers[y_real[i]*num_of_test+j,y_hat[i]] += 1

In [299]:
#yy_hat是将y_numbers里每行最大的那个数字所在的位置输出出来，也就是大片段的预测值
yy_hat = [np.argmax(y_numbers)for y_numbers in y_numbers]
#按照上一运算格中的排序方式，大片段真实值yy_real应该按下述方式构造（0，0。。0，1，1。。，1，2。。。）
yy_real= np.zeros(num_of_test*num_of_class)
for i in range(num_of_class):
    for j in range(num_of_test):
        yy_real[i*num_of_test+j] = i
#yy_number是大片段对比正确的数量
yy_number = 0
for i in range(len(yy_real)):
    if yy_hat[i]==yy_real[i]:
        yy_number+=1
print('最终分类正确率是',yy_number/len(yy_real))

最终分类正确率是 0.9375


In [300]:
#给出错误的序号及判定
for i in range(len(yy_hat)):
    if yy_hat[i] != yy_real[i]:
        print(name[int(yy_real[i])],test_set[i%num_of_test],'被判定成了',name[int(yy_hat[i])])

二胡 130 被判定成了 钢琴
古筝 125 被判定成了 琵琶
吉他 139 被判定成了 大提琴
吉他 146 被判定成了 大提琴
大提琴 102 被判定成了 小提琴
大提琴 103 被判定成了 吉他
小提琴 100 被判定成了 大提琴
小提琴 101 被判定成了 大提琴
小提琴 102 被判定成了 大提琴
小提琴 105 被判定成了 大提琴
小提琴 107 被判定成了 大提琴
琵琶 132 被判定成了 古筝
西塔尔琴 127 被判定成了 大提琴
西塔尔琴 137 被判定成了 小提琴
西塔尔琴 145 被判定成了 古筝
钢琴 107 被判定成了 古筝
钢琴 119 被判定成了 小提琴
钢琴 120 被判定成了 小提琴
钢琴 121 被判定成了 小提琴
钢琴 122 被判定成了 古筝
钢琴 123 被判定成了 小提琴
钢琴 124 被判定成了 古筝
钢琴 125 被判定成了 古筝
钢琴 126 被判定成了 古筝
钢琴 127 被判定成了 小提琴


In [301]:
test_matrix = np.zeros([num_of_class,num_of_class])
for i in range(len(yy_hat)):
        test_matrix[int(yy_real[i]),int(yy_hat[i])] += 1
print(test_matrix)

[[49.  0.  0.  0.  0.  0.  0.  1.]
 [ 0. 49.  0.  0.  0.  1.  0.  0.]
 [ 0.  0. 48.  2.  0.  0.  0.  0.]
 [ 0.  0.  1. 48.  1.  0.  0.  0.]
 [ 0.  0.  0.  5. 45.  0.  0.  0.]
 [ 0.  1.  0.  0.  0. 49.  0.  0.]
 [ 0.  1.  0.  1.  1.  0. 47.  0.]
 [ 0.  5.  0.  0.  5.  0.  0. 40.]]
